In [1]:
!nvidia-smi

Sun Mar  2 20:20:09 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:01:00.0 Off |                    0 |
| N/A   27C    P0              51W / 400W |      4MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Package Loading & Definition

In [1]:
import MOICVAE.SNF as snf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchmetrics
from torchmetrics import MeanSquaredError
from sklearn.impute import KNNImputer
import scripts
from functools import lru_cache
import optuna
from sklearn.metrics import pairwise_distances
from sklearn.exceptions import DataConversionWarning
import warnings
from torch.utils.data import Dataset
# 경고 무시 설정
warnings.filterwarnings("ignore", category=DataConversionWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launc

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launc

AttributeError: _ARRAY_API not found

/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)


In [2]:
# dataset for sim_matrix
class OmicsDataset_dict(Dataset): 
    def __init__(self, omic_dict, drug_dict, data): 
        self.omic_dict = omic_dict
        self.drug_dict = drug_dict
        self.cell_mapped_ids = {key:i for i, key in enumerate(self.omic_dict.keys())}
        # omic_dict의 키를 고유한 인덱스로 매핑
        # enumerate는 키들을 순서대로 열거하여 (인덱스, 키) 형태의 튜플로 반환
        # 딕셔너리 컴프레헨션: 각 키를 key로, 각 키의 인덱스를 i로 사용하여 {key:i}형태로 매핑된 딕셔너리 만듬.
        self.drug_mapped_ids = {key:i for i, key in enumerate(self.drug_dict.keys())}
        self.data = data
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx): # idx = train_data
        instance = self.data.iloc[idx] 
        cell_id = instance.iloc[0]
        drug_id = instance.iloc[1]
        target = instance.iloc[2]
        
        #omics_data = { # usage of dictionary here causes a problem or crash with collate_fn function in Dataloader 
        #    cell_id : {
        #        data_type: self.omic_dict[cell_id][data_type] for data_type in self.omic_dict[cell_id].keys()
        #    }
        #}
        
        return (torch.cat([self.omic_dict[cell_id][modality] for modality in self.omic_dict[cell_id].keys()]), 
                self.drug_dict[drug_id],
                torch.Tensor([target]),
                torch.Tensor([self.cell_mapped_ids[cell_id]]),
                torch.Tensor([self.drug_mapped_ids[drug_id]])) 

In [3]:
# MDA model
class MultimodalAutoencoder(nn.Module):
    def __init__(self, input_dims,  hidden_dim_encoders = 150, embed_dim = 75, fusion_dim = 150, dropout_encoders = 0.2):
        # get input as a dictionary
        super(MultimodalAutoencoder, self).__init__()
        # EEEEEEEEEEncoder
        self.input_dims = input_dims
        print(f"Expected input_dims: {len(input_dims)}")
        self.num_modalities = len(input_dims)
        self.do = nn.Dropout(dropout_encoders)

        self.omics_encoder = nn.ModuleList([
            nn.Sequential(
                nn.Linear(mod_input_dim, hidden_dim_encoders), # input 
                nn.ReLU(),
                nn.Linear(hidden_dim_encoders, embed_dim) # encoder hidden layer: 150, 75 as the value from the paper. so we start from this 
            )                                 # I dont get why they used 150, 75 for dimension, but we can tune it later
            for mod_input_dim in self.input_dims
        ])
        # fused latent feature 
        self.fusion_layer = nn.Sequential( # I think we need a fusion layer here, to combine the data modalities
            nn.Linear(embed_dim * self.num_modalities, fusion_dim),
            nn.ReLU(),
            nn.Linear(fusion_dim, embed_dim) # This concatenate latent features of all omics data, and fusion them and make its dim final latent dim
        )                                     # This is the only way I can think of to fuse omics data
        # decoder
        self.omics_decoder = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim_encoders),
            nn.ReLU(),
            nn.Linear(hidden_dim_encoders, sum(input_dims))
        )
        # I actually dont understand this step in paper. they said that decoder has symmetric structure as encoder,
        # but the data after MDA they provided, has weird dimension(363x90) which makes no sense. this is the point that i cant understand

    def forward(self, x):
        latent_features = [] # get dictionary as an input 
        start_idx = 0
        for i, encoder in enumerate(self.omics_encoder):
            mod_input_dim = self.input_dims[i]
            x_modality = x[:, start_idx:start_idx + mod_input_dim]
            latent_features.append(encoder(self.do(x_modality)))
            start_idx += mod_input_dim
            
        latent_fused = torch.cat(latent_features, dim=1)
        latent_final = self.fusion_layer(latent_fused)
        decoded = self.omics_decoder(latent_final)
        return decoded, latent_final

In [4]:
# Main resnet model
class ResNet(nn.Module):
    def __init__(self, embed_dim=256, hidden_dim=1024, dropout=0.1, n_layers=6, norm="layernorm"):
        super().__init__()
        self.mlps = nn.ModuleList()
        
        # Determine normalization layer
        if norm == "layernorm":
            norm_layer = nn.LayerNorm
        elif norm == "batchnorm":
            norm_layer = nn.BatchNorm1d
        else:
            norm_layer = nn.Identity
        
        # Create MLP layers
        for _ in range(n_layers):
            self.mlps.append(
                nn.Sequential(
                    nn.Linear(embed_dim, hidden_dim),
                    norm_layer(hidden_dim),
                    nn.ReLU(),
                    nn.Dropout(dropout),
                    nn.Linear(hidden_dim, embed_dim)
                )
            )
        
        self.lin = nn.Linear(embed_dim, 1)  # Output layer

    def forward(self, x):
        for layer in self.mlps:
            
            x = (layer(x) + x) / 2  # Residual connection
        return self.lin(x)


class Main_model(nn.Module):
    def __init__(self, embed_dim=256, hidden_dim=1024, dropout=0.1, n_layers=6, norm="layernorm", 
                 dropout_omics=0.4, dropout_omics_finetuning=0.4):
        super().__init__()
        
        # ResNet as backbone
        self.resnet = ResNet(embed_dim, hidden_dim, dropout, n_layers, norm)
        
        # Modified embed_d: Two-layer MLP with dropout after ReLU
        self.embed_d = nn.Sequential(
            nn.LazyLinear(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, embed_dim)
        )
        
        # Modified embed_c: Two-layer MLP with dropout after ReLU and before first Linear Layer
        self.embed_c = nn.Sequential(
            nn.Dropout(dropout_omics_finetuning),
            nn.LazyLinear(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_omics),
            nn.Linear(hidden_dim, embed_dim)
        )

    def forward(self, c, d):
        # Combine embedded inputs and pass through ResNet
        return self.resnet(self.embed_d(d) + self.embed_c(c))


In [5]:
# get Data function
@lru_cache(maxsize=None)
def get_data_corr(n_fold = 0, fp_radius = 2, transform_into_corr = True, typ = ["rnaseq", "mutations", "cnvs"],
                  #reconstructed = None
                 ):
    # drug
    smile_dict = pd.read_csv("data/smiles.csv", index_col=0)
    fp = scripts.FingerprintFeaturizer(R = fp_radius)
    drug_dict = fp(smile_dict.iloc[:, 1], smile_dict.iloc[:, 0])
    
    # loading all datasets
    driver_genes = pd.read_csv("data/driver_genes.csv").loc[:, "symbol"].dropna()

    rnaseq = pd.read_csv("data/rnaseq_normcount.csv", index_col=0)
    driver_columns = rnaseq.columns.isin(driver_genes)
    filtered_rna = rnaseq.loc[:, driver_columns]
    
    proteomics = pd.read_csv("data/proteomics.csv", index_col=0)
    
    mutation = pd.read_csv("data/binary_mutations.csv")
    mutation.columns = mutation.iloc[0]
    mutation = mutation.iloc[2:,:].set_index("gene_symbol")
    driver_columns = mutation.columns.isin(driver_genes)
    filtered_mut = mutation.loc[:, driver_columns]
    filtered_mut = filtered_mut.astype(float)

    methylations = pd.read_csv("data/methylations.csv",index_col = 0).sort_index(ascending = True)

    cnvs = pd.read_csv("data/copy_number_variations.csv",index_col= 0)

    # concatenate all dataset 
    # inner join based on index: model_ids with NaN are automatically filtered out 
    data_concat = pd.concat([filtered_rna, proteomics, filtered_mut, methylations, cnvs], axis=1, join='inner')
    
    
    # Filter data by common indices in all modalities
    filtered_rna = filtered_rna[filtered_rna.index.isin(data_concat.index)]
    proteomics = proteomics[proteomics.index.isin(data_concat.index)]
    filtered_mut = filtered_mut[filtered_mut.index.isin(data_concat.index)]
    methylations = methylations[methylations.index.isin(data_concat.index)]
    cnvs = cnvs[cnvs.index.isin(data_concat.index)]
    
    # Initialize cell_dict
    cell_dict = {}

    if not transform_into_corr : #and reconstructed is None:

        dims = []
        if "rnaseq" in typ:
            dims.append(filtered_rna.shape[1])
        if "proteomics" in typ:
            dims.append(proteomics.shape[1])
        if "mutations" in typ:
            dims.append(filtered_mut.shape[1])
        if "methylations" in typ:
            dims.append(methylations.shape[1])
        if "cnvs" in typ:
            dims.append(cnvs.shape[1])
        
        for cell in data_concat.index:
            # Initialize a sub-dictionary for each cell
            concatenated_data = []
            
            # Add data for each type specified in typ
            if "rnaseq" in typ:
                concatenated_data.append(filtered_rna.loc[cell].to_numpy())
            if "proteomics" in typ:
                concatenated_data.append(proteomics.loc[cell].to_numpy())
            if "mutations" in typ:
                concatenated_data.append(filtered_mut.loc[cell].to_numpy())
            if "methylations" in typ:
                concatenated_data.append(methylations.loc[cell].to_numpy())
            if "cnvs" in typ:
                concatenated_data.append(cnvs.loc[cell].to_numpy())

            cell_dict[cell] = torch.Tensor(np.concatenate(concatenated_data))
            
#    if reconstructed is not None:
#        for cell_idx, cell in enumerate(data_concat.index):
#            # cell_dict에 reconstructed 텐서의 각 행(cell 데이터) 저장
#            cell_dict[cell] = reconstructed[cell_idx]
        

    # GDSC
    GDSC1 = pd.read_csv("data/GDSC1.csv", index_col=0)
    # default, remove data where lines or drugs are missing:
    data = GDSC1.query("SANGER_MODEL_ID in @data_concat.index & DRUG_ID in @drug_dict.keys()")
    unique_cell_lines = data.loc[:, "SANGER_MODEL_ID"].unique()

    np.random.seed(420) # for comparibility, don't change it!
    np.random.shuffle(unique_cell_lines)
    folds = np.array_split(unique_cell_lines, 10)
    test_lines = folds[0] 
    train_idxs = list(range(10))
    train_idxs.remove(n_fold)
    np.random.seed(420)
    validation_idx = np.random.choice(train_idxs)
    train_idxs.remove(validation_idx)
    train_lines = np.concatenate([folds[idx] for idx in train_idxs])
    validation_lines = folds[validation_idx]
    test_lines = folds[n_fold] 

        # no change needed, query works fine with some missing
    train_data = data.query("SANGER_MODEL_ID in @train_lines")
    validation_data = data.query("SANGER_MODEL_ID in @validation_lines")
    test_data = data.query("SANGER_MODEL_ID in @test_lines")
    
    if transform_into_corr: #and reconstructed is None:
        # train, val, test among filtered data
        # these are valid train_, val_ and test_data index
        
        
        n_train = len(train_lines)  
        n_val = len(validation_lines)      
        n_test = len(test_lines)
        
        # Precompute similarity matrices for each data type
        similarity_matrices = {}
        dims = []
        
        if "rnaseq" in typ:
            exp_com = np.corrcoef(np.vstack([filtered_rna.loc[train_lines], 
                                             filtered_rna.loc[validation_lines], 
                                             filtered_rna.loc[test_lines]]), rowvar=True)
            train = exp_com[:n_train, :n_train]
            val = exp_com[n_train:n_train+n_val, :n_train]
            test = exp_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["rnaseq"] = sim_combined
            dims.append(len(similarity_matrices["rnaseq"][0]))
        
        if "proteomics" in typ:
            prot_com = np.corrcoef(np.vstack([proteomics.loc[train_lines], 
                                              proteomics.loc[validation_lines], 
                                              proteomics.loc[test_lines]]), rowvar=True)
            train = prot_com[:n_train, :n_train]
            val = prot_com[n_train:n_train+n_val, :n_train]
            test = prot_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["proteomics"] = sim_combined
            dims.append(len(similarity_matrices["proteomics"][0]))
        
        if "mutations" in typ:
            train_snp = filtered_mut.loc[train_lines].astype(bool)
            val_snp = filtered_mut.loc[validation_lines].astype(bool)
            test_snp = filtered_mut.loc[test_lines].astype(bool)
            
            train = 1 - pairwise_distances(train_snp.values, train_snp.values, metric="jaccard")
            val = 1 - pairwise_distances(val_snp.values, train_snp.values, metric="jaccard")
            test = 1 - pairwise_distances(test_snp.values, train_snp.values, metric="jaccard")
    
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["mutations"] = sim_combined
            dims.append(len(similarity_matrices["mutations"][0]))
        
        if "methylations" in typ:
            methyl_com = np.nan_to_num(np.corrcoef(np.vstack([methylations.loc[train_lines], 
                                                methylations.loc[validation_lines], 
                                                methylations.loc[test_lines]]), rowvar=True))
            train = methyl_com[:n_train, :n_train]
            val = methyl_com[n_train:n_train+n_val, :n_train]
            test = methyl_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["methylations"] = sim_combined
            dims.append(len(similarity_matrices["methylations"][0]))
        
        if "cnvs" in typ:
            cnv_com = np.nan_to_num(np.corrcoef(np.vstack([cnvs.loc[train_lines], # nan-generation problem fixed 
                                             cnvs.loc[validation_lines], 
                                             cnvs.loc[test_lines]]), rowvar=True))
            train= cnv_com[:n_train, :n_train]
            val= cnv_com[n_train:n_train+n_val, :n_train]
            test= cnv_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["cnvs"] = sim_combined
            dims.append(len(similarity_matrices["cnvs"][0]))
            
        cell_dict = {}

        # 
        for cell in unique_cell_lines:
            cell_dict[cell] = {}
            for data_type in typ:
                sim_matrices = similarity_matrices[data_type]
                sim_tensor = torch.Tensor(sim_matrices)
                cell_idx = np.where(unique_cell_lines == cell)[0][0]
                cell_dict[cell][data_type] = sim_tensor[cell_idx]
    
        return (OmicsDataset_dict(cell_dict, drug_dict, train_data),
        OmicsDataset_dict(cell_dict, drug_dict, validation_data),
        OmicsDataset_dict(cell_dict, drug_dict, test_data),
        dims)

    return (scripts.OmicsDataset(cell_dict, drug_dict, train_data),
    scripts.OmicsDataset(cell_dict, drug_dict, validation_data),
    scripts.OmicsDataset(cell_dict, drug_dict, test_data),
    dims)
    

In [6]:
# model training function
def train_step(model, optimizer, loader, config, device):
    loss = nn.MSELoss()
    ls = []
    model.train()
    for x in loader:
        optimizer.zero_grad()
        out = model(x[0].to(device), x[1].to(device))
        l = loss(out.squeeze(), x[2].to(device).squeeze())
        l.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), config["optimizer"]["clip_norm"])
        ls += [l.item()]
        optimizer.step()
    return np.mean(ls)

# finetune old version: uses pre-trained weights, which is non-sense.
"""
def finetune_model_train(config, train_dataset, pre_trained_weight_path, validation_dataset=None, use_momentum=True, callback_epoch = None):
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=config["optimizer"]["batch_size"],
                                           drop_last=True,
                                          shuffle=True)
    if validation_dataset is not None:
        val_loader = torch.utils.data.DataLoader(validation_dataset,
                                               batch_size=config["optimizer"]["batch_size"],
                                               drop_last=False,
                                              shuffle=False)
        
    model = Main_model(embed_dim=config["model"]["embed_dim"],
                       hidden_dim=config["model"]["hidden_dim"], 
                       dropout=config["model"]["dropout"], 
                       n_layers=config["model"]["n_layers"],  
                       dropout_omics=config["model"]["dropout_omics"], 
                       dropout_omics_finetuning=config["model"]["dropout_omics_finetuning"],
                       norm=config["model"]["norm"])
    model.embed_c.load_state_dict(torch.load(pre_trained_weight_path), strict=False) # pre-trained weights
    optimizer = torch.optim.Adam(model.parameters(), config["optimizer"]["learning_rate"])
    device = torch.device(config["env"]["device"])
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=5)
    early_stop = scripts.EarlyStop(config["optimizer"]["stopping_patience"])
    model.to(device)
    metrics = torchmetrics.MetricTracker(torchmetrics.MetricCollection(
    {"R_cellwise_residuals":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="drugs",
                          average="macro",
                          residualize=True),
    "R_cellwise":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="cell_lines",
                          average="macro",
                          residualize=False),
    "MSE":torchmetrics.MeanSquaredError()}))
    metrics.to(device)
    for epoch in range(config["env"]["max_epochs"]):
        train_loss = train_step(model, optimizer, train_loader, config, device)
        if epoch == 0:  # first epoch
            # if main model is called with input data, then LazyLinear is initialized with input
            # our pretrained weights has already [hidden_dim, embed_dim] dimension, so it cannot be directly loaded  into model
            # but embed_c will eventually be [hidden_dim, embed_dim] dimension after last linear layer, so we override the weights after first epoch
            optimizer = torch.optim.Adam(model.parameters(), lr=config["optimizer"]["learning_rate"]) # reinitialize to prevent any problem
            
        lr_scheduler.step(train_loss)
        
        if validation_dataset is not None:
            validation_metrics = scripts.evaluate_step(model,val_loader, metrics, device)
            if epoch > 0 & use_momentum:
                val_target = 0.2*val_target + 0.8*validation_metrics['R_cellwise_residuals']
            else:
                val_target = validation_metrics['R_cellwise_residuals']
        else:
            val_target = None
        if callback_epoch is None:
            print(f"epoch : {epoch}: train loss: {train_loss} Smoothed R interaction (validation) {val_target}")
        else:
            callback_epoch(epoch, val_target)
        if early_stop(train_loss):
            break
    return val_target, model
"""

def finetune_model_train(config, train_dataset, pre_trained_model, validation_dataset=None, use_momentum=True, callback_epoch=None):
    device = torch.device(config["env"]["device"])
    pre_raw_model.eval()
    latent_dict = {}

    with torch.no_grad():
        for cell_id, omics_tensor in fine_raw_train_dataset.omic_dict.items():
            omics_tensor = omics_tensor.to(device)  # GPU로 이동
            
            #  Autoencoder의 forward() 호출하여 바로 latent_final 획득
            _, latent_final = pre_raw_model(omics_tensor.unsqueeze(0))  # decoded는 필요 없으므로 _ 처리
    
            latent_dict[cell_id] = latent_final.squeeze(0).cpu()  # Latent Representation 저장
    
    train_dataset.omic_dict = latent_dict
    print(f"the omics data of {len(latent_dict)} samples are replaced to pre-trained latent features.")
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config["optimizer"]["batch_size"],
        drop_last=True,
        shuffle=True
    )

    if validation_dataset is not None:
        val_loader = torch.utils.data.DataLoader(
            validation_dataset,
            batch_size=config["optimizer"]["batch_size"],
            drop_last=False,
            shuffle=False
        )

    ### Fine-tuning 모델 생성 ###
    model = Main_model(
        embed_dim=config["model"]["embed_dim"],
        hidden_dim=config["model"]["hidden_dim"], 
        dropout=config["model"]["dropout"], 
        n_layers=config["model"]["n_layers"],  
        dropout_omics=config["model"]["dropout_omics"], 
        dropout_omics_finetuning=config["model"]["dropout_omics_finetuning"],
        norm=config["model"]["norm"]
    )

    optimizer = torch.optim.Adam(model.parameters(), config["optimizer"]["learning_rate"])
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=5)
    early_stop = scripts.EarlyStop(config["optimizer"]["stopping_patience"])
    model.to(device)

    metrics = torchmetrics.MetricTracker(torchmetrics.MetricCollection({
        "R_cellwise_residuals": scripts.GroupwiseMetric(
            metric=torchmetrics.functional.pearson_corrcoef,
            grouping="drugs",
            average="macro",
            residualize=True
        ),
        "R_cellwise": scripts.GroupwiseMetric(
            metric=torchmetrics.functional.pearson_corrcoef,
            grouping="cell_lines",
            average="macro",
            residualize=False
        ),
        "MSE": torchmetrics.MeanSquaredError()
    }))
    metrics.to(device)

    ### Fine-tuning Loop ###
    for epoch in range(config["env"]["max_epochs"]):
        train_loss = train_step(model, optimizer, train_loader, config, device)

        if epoch == 0:  
            optimizer = torch.optim.Adam(model.parameters(), lr=config["optimizer"]["learning_rate"])
        
        lr_scheduler.step(train_loss)
        
        if validation_dataset is not None:
            validation_metrics = scripts.evaluate_step(model, val_loader, metrics, device)
            if epoch > 0 and use_momentum:
                val_target = 0.2 * val_target + 0.8 * validation_metrics['R_cellwise_residuals']
            else:
                val_target = validation_metrics['R_cellwise_residuals']
        else:
            val_target = None

        if callback_epoch is None:
            print(f"epoch : {epoch}: train loss: {train_loss} Smoothed R interaction (validation) {val_target}")
        else:
            callback_epoch(epoch, val_target)

        if early_stop(train_loss):
            break

    return val_target, model


In [7]:
# pre-training 과정 이후, weights를 반환하는 것이 아닌 latnet feature를 반환하는 함수 

def pre_train_step(config, pre_dataset, input_dims):
    """
    Pre-training function for Multimodal Autoencoder.
    This function can handle both raw and similarity-based datasets.
    
    Parameters:
        - config: Training configuration dictionary
        - pre_dataset: Dataset to train the autoencoder (can be raw or similarity-based)
        - input_dims: Input dimensions for the autoencoder

    Returns:
        - pre_trained_model: Trained autoencoder model
        - total_losses: List of training losses per epoch
    """
    # 1️⃣ 데이터 로더 설정
    pre_train_loader = torch.utils.data.DataLoader(
        pre_dataset,
        batch_size=config["optimizer"]["pre_batch_size"],
        shuffle=True,
        drop_last=True
    )

    # 2️⃣ 모델 생성
    pre_model = MultimodalAutoencoder(
        input_dims=input_dims,
        hidden_dim_encoders=config["model"]["hidden_dim_encoders"],
        embed_dim=config["model"]["embed_dim"],
        fusion_dim=config["model"]["fusion_dim"],
        dropout_encoders=config["model"]["dropout_encoders"]
    )

    # 3️⃣ 옵티마이저, 손실 함수, 스케줄러 설정
    optimizer = torch.optim.Adam(pre_model.parameters(), lr=config["optimizer"]["lr_pretraining"])
    criterion = nn.MSELoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)
    device = torch.device(config["env"]["device"])

    pre_model.to(device)
    pre_model.train()

    pre_training_epochs = config["model"]["pre_training_epochs"]
    total_losses = []

    # 4️⃣ 모델 학습
    for epoch in range(pre_training_epochs):
        total_loss = []
        
        for batch in pre_train_loader:
            inputs = batch[0].to(device)  # 데이터는 `batch[0]`에 저장됨

            optimizer.zero_grad()  # 기울기 초기화
            reconstructed, latent_features = pre_model(inputs)  # Latent features 추출

            # 손실 계산 (입력 데이터와 복원된 데이터 비교)
            loss = criterion(reconstructed, inputs)
            
            loss.backward()  # 기울기 계산
            optimizer.step()  # 옵티마이저 스텝 진행

            total_loss.append(loss.item())

        # 평균 손실값을 계산하고 스케줄러 적용
        train_loss = np.mean(total_loss)
        total_losses.append(train_loss)
        scheduler.step(train_loss)

        # 학습 상태 출력
        print(f'Epoch {epoch + 1}/{pre_training_epochs}, Loss: {train_loss:.5f}')

    print("Pre-training complete!")

    return pre_model, total_losses


    

In [8]:
# config
config = {"features" : {"fp_radius":2,
                        "use_correlation_representation": True,
                        "num_modalities": 4},
          "optimizer": {"batch_size": 220,
                        "clip_norm":19,
                        "learning_rate": 0.0004592646200179472,
                        "stopping_patience":15,
                        "pre_batch_size": 200,
                        "lr_pretraining": 0.0004592646200179472},
          "model":{"embed_dim":485, # shared
                 "hidden_dim":696, 
                 "dropout":0.48541242824674574, 
                 "n_layers": 4, 
                 "norm": "batchnorm", 
                 "hidden_dim_encoders": 256, # ENCODER
                 "fusion_dim": 700, # ENCODER
                 "dropout_encoders": 0.2,
                 "dropout_omics": 0.4, # second
                 "dropout_omics_finetuning": 0.4, # first
                 "pre_training_epochs": 100}, 
         "env": {"fold": 0,  
                "device":"cuda:0", 
                 "max_epochs": 100, 
                 "search_hyperparameters":False}} 

# Using Similarity Matrix 

## Pre-training

In [82]:
# data loading with corr
pre_sim_train_dataset, pre_sim_val_dataset, pre_sim_test_dataset, pre_sim_dims = get_data_corr(n_fold = config["env"]["fold"],
                                                           fp_radius = config["features"]["fp_radius"], 
                                                           transform_into_corr = config["features"]["use_correlation_representation"],
                                                           typ = ("rnaseq", "proteomics", "mutations", "methylations"))

[01:12:40] DEPRECATION WARNING: please use MorganGenerator
[01:12:40] DEPRECATION WARNING: please use MorganGenerator
[01:12:40] DEPRECATION WARNING: please use MorganGenerator
[01:12:40] DEPRECATION WARNING: please use MorganGenerator
[01:12:40] DEPRECATION WARNING: please use MorganGenerator
[01:12:40] DEPRECATION WARNING: please use MorganGenerator
[01:12:40] DEPRECATION WARNING: please use MorganGenerator
[01:12:40] DEPRECATION WARNING: please use MorganGenerator
[01:12:40] DEPRECATION WARNING: please use MorganGenerator
[01:12:40] DEPRECATION WARNING: please use MorganGenerator
[01:12:40] DEPRECATION WARNING: please use MorganGenerator
[01:12:40] DEPRECATION WARNING: please use MorganGenerator
[01:12:40] DEPRECATION WARNING: please use MorganGenerator
[01:12:40] DEPRECATION WARNING: please use MorganGenerator
[01:12:40] DEPRECATION WARNING: please use MorganGenerator
[01:12:40] DEPRECATION WARNING: please use MorganGenerator
[01:12:40] DEPRECATION WARNING: please use MorganGenerat

In [13]:
# pre training with sim_matrix
pre_sim_train_loader = torch.utils.data.DataLoader(
        pre_sim_train_dataset,
        batch_size = config["optimizer"]["pre_batch_size"],
        shuffle=True,
        drop_last=True
    )

#if validation_dataset is not None:
#    val_loader = torch.utils.data.DataLoader(val_dataset,batch_size = batch_size,shuffle=True,drop_last=True)
    
sample_batch = next(iter(pre_sim_train_loader))
sample_omics_data = sample_batch[0]  # 첫 번째 요소는 omics_data (dict)
mod_len = sample_omics_data.shape[1] // config["features"]["num_modalities"]
#input_dict = {f"modality_{i+1}": mod_len for i in range(num_modality)}

pre_sim_model = MultimodalAutoencoder(input_dims = pre_sim_dims, 
                              hidden_dim_encoders = config["model"]["hidden_dim_encoders"],
                              embed_dim = config["model"]["embed_dim"],
                              fusion_dim = config["model"]["fusion_dim"],
                              #num_modalities = config["features"]["num_modalities"], 
                              dropout_encoders = config["model"]["dropout_encoders"])

optimizer = torch.optim.Adam(pre_sim_model.parameters(), lr=config["optimizer"]["lr_pretraining"])
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pre_sim_model.to(device)

# model training
for epoch in range(config["model"]["pre_training_epochs"]):
    total_loss = []
    pre_sim_model.train()


#    if validation_dataset is not None:
#        validation_metrics = evaluate_step(model,val_loader, metrics, device)
#        if epoch > 0 & use_momentum:
#            val_target = 0.2*val_target + 0.8*validation_metrics['R_cellwise_residuals']
#        else:
#            val_target = validation_metrics['R_cellwise_residuals']

    
    # train step
    for batch in pre_sim_train_loader:
        inputs = batch[0].to(device)

        optimizer.zero_grad() # gradient initialization
        reconstructed, latent = pre_sim_model(inputs)

        #target_data = torch.cat([inputs[modality] for modality in inputs], dim=1)
        # I think, here inappropriate loss is used. 
        # From what I know, Autoencoder reconstructs input feature, and calculate loss through comparing input feature and reconstructed feature.
        # So, calculating MSE comparing with target_data, which is drug sensitivity, is nonsense. 
        #loss = criterion(reconstructed, target_data)
        
        # here is a new loss function, but it still outputs nan value as a loss...
        loss = criterion(reconstructed, inputs)
        
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=config["optimizer"]["clip_norm"])
        optimizer.step()

        total_loss.append(loss.item())
        
    train_loss = np.mean(total_loss)
    scheduler.step(train_loss)

    print(f'Epoch {epoch + 1}/{config["model"]["pre_training_epochs"]}, Loss: {train_loss:.5f}')

torch.save(pre_sim_model.fusion_layer.state_dict(), "trained_models/pretrained_omics.pth") # save pre-trained weights from fusion layer
#torch.save(reconstructed,"reconstructedMDA_data.pth")
print("Training complete!")


Expected input_dims: 4
Epoch 1/100, Loss: 0.00718
Epoch 2/100, Loss: 0.00167
Epoch 3/100, Loss: 0.00127
Epoch 4/100, Loss: 0.00104
Epoch 5/100, Loss: 0.00091
Epoch 6/100, Loss: 0.00082
Epoch 7/100, Loss: 0.00076
Epoch 8/100, Loss: 0.00070
Epoch 9/100, Loss: 0.00065
Epoch 10/100, Loss: 0.00060
Epoch 11/100, Loss: 0.00057
Epoch 12/100, Loss: 0.00053
Epoch 13/100, Loss: 0.00050
Epoch 14/100, Loss: 0.00048
Epoch 15/100, Loss: 0.00045
Epoch 16/100, Loss: 0.00043
Epoch 17/100, Loss: 0.00041
Epoch 18/100, Loss: 0.00040
Epoch 19/100, Loss: 0.00038
Epoch 20/100, Loss: 0.00037
Epoch 21/100, Loss: 0.00036
Epoch 22/100, Loss: 0.00036
Epoch 23/100, Loss: 0.00035
Epoch 24/100, Loss: 0.00034
Epoch 25/100, Loss: 0.00034
Epoch 26/100, Loss: 0.00033
Epoch 27/100, Loss: 0.00033
Epoch 28/100, Loss: 0.00032
Epoch 29/100, Loss: 0.00032
Epoch 30/100, Loss: 0.00032
Epoch 31/100, Loss: 0.00031
Epoch 32/100, Loss: 0.00031
Epoch 33/100, Loss: 0.00031
Epoch 34/100, Loss: 0.00031
Epoch 35/100, Loss: 0.00031
Epoch 

In [ ]:
pretrained_sim_model, pretraining_sim_losses = pre_train_step(config, torch.utils.data.ConcatDataset([pre_sim_train_dataset, pre_sim_val_dataset]), input_dims = pre_sim_dims)

Expected input_dims: 4
Epoch 1/100, Loss: 0.00649
Epoch 2/100, Loss: 0.00156
Epoch 3/100, Loss: 0.00117
Epoch 4/100, Loss: 0.00099
Epoch 5/100, Loss: 0.00088
Epoch 6/100, Loss: 0.00080
Epoch 7/100, Loss: 0.00074
Epoch 8/100, Loss: 0.00068
Epoch 9/100, Loss: 0.00064
Epoch 10/100, Loss: 0.00059
Epoch 11/100, Loss: 0.00056
Epoch 12/100, Loss: 0.00053
Epoch 13/100, Loss: 0.00050
Epoch 14/100, Loss: 0.00047
Epoch 15/100, Loss: 0.00045
Epoch 16/100, Loss: 0.00043
Epoch 17/100, Loss: 0.00042
Epoch 18/100, Loss: 0.00040
Epoch 19/100, Loss: 0.00039
Epoch 20/100, Loss: 0.00038
Epoch 21/100, Loss: 0.00037
Epoch 22/100, Loss: 0.00036
Epoch 23/100, Loss: 0.00035
Epoch 24/100, Loss: 0.00035
Epoch 25/100, Loss: 0.00034
Epoch 26/100, Loss: 0.00034
Epoch 27/100, Loss: 0.00033
Epoch 28/100, Loss: 0.00033
Epoch 29/100, Loss: 0.00033
Epoch 30/100, Loss: 0.00032
Epoch 31/100, Loss: 0.00032
Epoch 32/100, Loss: 0.00032
Epoch 33/100, Loss: 0.00032
Epoch 34/100, Loss: 0.00032
Epoch 35/100, Loss: 0.00032
Epoch 

In [ ]:
# evaluation MDA

pre_sim_eval_loader = torch.utils.data.DataLoader(
        pre_sim_test_dataset,
        batch_size = config["optimizer"]["pre_batch_size"],
        shuffle=False, # we dont need this in evaluation step => repeatable result, for comparision
        drop_last=False # to use all data
    )

pre_sim_model.to(device)
pre_sim_model.eval()

eval_losses = []

for x in pre_sim_eval_loader: 
    with torch.no_grad(): # no autograd => in evaluation step we dont need to update weights
        inputs = batch[0].to(device)
        reconstructed, _ = pre_sim_model(inputs)
        evel_loss = criterion(reconstructed,inputs)
        eval_losses.append(evel_loss.item())

final_eval_loss = np.mean(eval_losses)
print(f"Final MSE of Model: {final_eval_loss:.5f}")
        

# Model Training & Evaluation

In [ ]:
#recon_sim = torch.load("reconstructedMDA_data.pth")
fine_sim_train_dataset, fine_val_sim_dataset, fine_test_sim_dataset, dim = get_data_corr(n_fold = config["env"]["fold"],
                                                           fp_radius = config["features"]["fp_radius"], 
                                                           transform_into_corr = config["features"]["use_correlation_representation"],
                                                           typ = ("rnaseq", "proteomics", "mutations", "methylations"),
                                                           #reconstructed = recon_sim
                                                                                   )
# use sim_matrix as an input data as well 

In [19]:
#weight_path = "trained_models/pretrained_omics.pth"

_, main_sim_model = finetune_model_train(config,
                                         torch.utils.data.ConcatDataset([fine_sim_train_dataset, fine_val_sim_dataset]), 
                                         pre_trained_weight_path = weight_path,
                                         validation_dataset = None, use_momentum=False)
metrics = torchmetrics.MetricTracker(torchmetrics.MetricCollection(
    {"R_cellwise_residuals":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="drugs",
                          average="macro",
                          residualize=True),
    "R_cellwise":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="cell_lines",
                          average="macro",
                          residualize=False),
    "MSE":torchmetrics.MeanSquaredError()}))
metrics.to(device)
fine_sim_test_dataloader = torch.utils.data.DataLoader(fine_test_sim_dataset,
                                       batch_size=config["optimizer"]["batch_size"],
                                       drop_last=False,
                                      shuffle=False,
                                      pin_memory=True)

/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: FutureWarning: The default value for `maximize` will be changed from `True` to `None` in v1.7.0 of TorchMetrics,will automatically infer the value based on the `higher_is_better` attribute of the metric (if such attribute exists) or raise an error if it does not. If you are explicitly setting the `maximize` argument to either `True` or `False` already, you can ignore this warning.
  warnings.warn(*args, **kwargs)  # noqa: B028


Pre-trained weights applied to the final Linear layer!
epoch : 0: train loss: 2.3652852571791567 Smoothed R interaction (validation) None
epoch : 1: train loss: 2.0498048929113097 Smoothed R interaction (validation) None
epoch : 2: train loss: 1.9744003243440456 Smoothed R interaction (validation) None
epoch : 3: train loss: 1.923729983336846 Smoothed R interaction (validation) None
epoch : 4: train loss: 1.8867335819048698 Smoothed R interaction (validation) None
epoch : 5: train loss: 1.8523730247365529 Smoothed R interaction (validation) None
epoch : 6: train loss: 1.8281392441102395 Smoothed R interaction (validation) None
epoch : 7: train loss: 1.8046435273916672 Smoothed R interaction (validation) None
epoch : 8: train loss: 1.7864722365649286 Smoothed R interaction (validation) None
epoch : 9: train loss: 1.7648730460733948 Smoothed R interaction (validation) None
epoch : 10: train loss: 1.7477140310226176 Smoothed R interaction (validation) None
epoch : 11: train loss: 1.734586

In [20]:
main_sim_final_metrics_nohyper = scripts.evaluate_step(main_sim_model, fine_sim_test_dataloader, metrics, device, save_predictions = True, model_name = "pretrained_NoOptHyper", dataset_name = "sim_matrix")
print(f"main model final metrics: {main_sim_final_metrics_nohyper}")

/work/kim14/project_work/scripts/models.py:69: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525552843/work/aten/src/ATen/native/TensorShape.cpp:3277.)
  return torch.linalg.solve(A, Xy).T


Predictions saved to: results/pred_pretrained_NoOptHyper_sim_matrix_20250225_13:59:59_0d96501d-5d73-4701-9051-f09e9c5398b6.csv
main model final metrics: {'MSE': 2.1859209537506104, 'R_cellwise': 0.8763778209686279, 'R_cellwise_residuals': 0.0007535024778917432}


# Using raw data
## Pre-training

할일:

1. 페드로와 토의. 기존에 하던 sim-sim, raw-raw 방식이 논리적으로 말이 안된다는것을 건의해보기. 


### 문제!

Autoencoder의 역할은 단순히 입력을 압축한 뒤, 다시 복원하는 것이다. 이때 생성된 weights를 이후 Resnet의 fine-tuning에 사용하는것은 그렇게 논리적으로 신빙성이 있어보이지 않는다.
또한, Autoencoder와 Resnet의 목적이 다르기 때문에, 예측 성능을 높이는데 도움이 되지 않을 가능성이 크다. 

그러나, Autoencoder는 입력 데이터에서 중요한 정보를 추출해내서 압축하는 과정을 거치기 때문에, 이렇게 압축된 feature들은 중요한 정보만을 가지고 있을 가능성이 크다. 이를 아예 finetune 과정에서 입력으로 제공하면, 더욱 논리적으로 타당하다. 

pre-trained weight로 결과가 나아지지 않는다는 것을 이미 파악했으니, 이번에는 pre-training과정에서 생성된 latent feature를 fine-tune의 1차 입력 데이터로 사용하는 것으로 하자. 

In [27]:
# data loading with raw data
pre_raw_train_dataset, pre_raw_val_dataset, pre_raw_test_dataset, pre_raw_dims= get_data_corr(n_fold = config["env"]["fold"],
                                                           fp_radius = config["features"]["fp_radius"], 
                                                           transform_into_corr = False,
                                                           typ = ("rnaseq", "proteomics", "mutations", "methylations"))

In [29]:
# pre training with raw data 
# 가중치 저장 버전
"""
pre_raw_train_loader = torch.utils.data.DataLoader(
        pre_raw_train_dataset,
        batch_size = config["optimizer"]["pre_batch_size"],
        shuffle=True,
        drop_last=True
    )

#if validation_dataset is not None:
#    val_loader = torch.utils.data.DataLoader(val_dataset,batch_size = batch_size,shuffle=True,drop_last=True)
    
sample_batch = next(iter(pre_raw_train_loader))
sample_omics_data = sample_batch[0]  # 첫 번째 요소는 omics_data (dict)
mod_len = sample_omics_data.shape[1] // config["features"]["num_modalities"]
#input_dict = {f"modality_{i+1}": mod_len for i in range(num_modality)}

pre_raw_model = MultimodalAutoencoder(input_dims = pre_raw_dims, 
                              hidden_dim_encoders = config["model"]["hidden_dim_encoders"],
                              embed_dim = config["model"]["embed_dim"],
                              fusion_dim = config["model"]["fusion_dim"],
                              #num_modalities = config["features"]["num_modalities"], 
                              dropout_encoders = config["model"]["dropout_encoders"])

optimizer = torch.optim.Adam(pre_raw_model.parameters(), lr=config["optimizer"]["lr_pretraining"])
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pre_raw_model.to(device)
use_momentum = True
# model training
for epoch in range(config["model"]["pre_training_epochs"]):
    total_loss = []
    pre_raw_model.train()
 
    # train step
    for batch in pre_raw_train_loader:
        inputs = batch[0].to(device)
        #print(f"Actual input data shape: {inputs.shape}")

        optimizer.zero_grad() # gradient initialization
        reconstructed, latent = pre_raw_model(inputs)

        #target_data = torch.cat([inputs[modality] for modality in inputs], dim=1)
        # I think, here inappropriate loss is used. 
        # From what I know, Autoencoder reconstructs input feature, and calculate loss through comparing input feature and reconstructed feature.
        # So, calculating MSE comparing with target_data, which is drug sensitivity, is nonsense. 
        #loss = criterion(reconstructed, target_data)
        
        # here is a new loss function, but it still outputs nan value as a loss...
        loss = criterion(reconstructed, inputs)
        
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=config["optimizer"]["clip_norm"])
        optimizer.step()

        total_loss.append(loss.item())
        
    train_loss = np.mean(total_loss)
    scheduler.step(train_loss)

    print(f'Epoch {epoch + 1}/{config["model"]["pre_training_epochs"]}, Train Loss: {train_loss:.5f}')

torch.save(pre_raw_model.fusion_layer.state_dict(), "trained_models/pretrained_raw_omics.pth") # save pre-trained weights from fusion layer
#torch.save(reconstructed,"reconstructedMDA_data.pth")
print("Training complete!")
"""

Expected input_dims: 4
Epoch 1/100, Loss: 0.50720
Epoch 2/100, Loss: 0.22842
Epoch 3/100, Loss: 0.19452
Epoch 4/100, Loss: 0.17042
Epoch 5/100, Loss: 0.15249
Epoch 6/100, Loss: 0.13720
Epoch 7/100, Loss: 0.12511
Epoch 8/100, Loss: 0.11496
Epoch 9/100, Loss: 0.10688
Epoch 10/100, Loss: 0.09995
Epoch 11/100, Loss: 0.09482
Epoch 12/100, Loss: 0.09108
Epoch 13/100, Loss: 0.08740
Epoch 14/100, Loss: 0.10341
Epoch 15/100, Loss: 0.08243
Epoch 16/100, Loss: 0.08148
Epoch 17/100, Loss: 0.08112
Epoch 18/100, Loss: 0.08070
Epoch 19/100, Loss: 0.08023
Epoch 20/100, Loss: 0.07970
Epoch 21/100, Loss: 0.07967
Epoch 22/100, Loss: 0.07910
Epoch 23/100, Loss: 0.07876
Epoch 24/100, Loss: 0.07855
Epoch 25/100, Loss: 0.07827
Epoch 26/100, Loss: 0.11180
Epoch 27/100, Loss: 0.07711
Epoch 28/100, Loss: 0.07656
Epoch 29/100, Loss: 0.07639
Epoch 30/100, Loss: 0.07634
Epoch 31/100, Loss: 0.07629
Epoch 32/100, Loss: 0.07620
Epoch 33/100, Loss: 0.07613
Epoch 34/100, Loss: 0.07609
Epoch 35/100, Loss: 0.07597
Epoch 

In [65]:
# pre training with raw data! final version. 
pretrained_model, pretraining_losses = pre_train_step(config, torch.utils.data.ConcatDataset([pre_raw_train_dataset, pre_raw_val_dataset]), input_dims = pre_raw_dims)

Expected input_dims: 4
Epoch 1/100, Train Loss: 0.47233
Epoch 2/100, Train Loss: 0.22022
Epoch 3/100, Train Loss: 0.18782
Epoch 4/100, Train Loss: 0.16429
Epoch 5/100, Train Loss: 0.14611
Epoch 6/100, Train Loss: 0.13125
Epoch 7/100, Train Loss: 0.11980
Epoch 8/100, Train Loss: 0.11032
Epoch 9/100, Train Loss: 0.10305
Epoch 10/100, Train Loss: 0.09690
Epoch 11/100, Train Loss: 0.09961
Epoch 12/100, Train Loss: 0.08784
Epoch 13/100, Train Loss: 0.08587
Epoch 14/100, Train Loss: 0.08438
Epoch 15/100, Train Loss: 0.08301
Epoch 16/100, Train Loss: 0.08191
Epoch 17/100, Train Loss: 0.08125
Epoch 18/100, Train Loss: 0.08143
Epoch 19/100, Train Loss: 0.07934
Epoch 20/100, Train Loss: 0.07947
Epoch 21/100, Train Loss: 0.07930
Epoch 22/100, Train Loss: 0.10375
Epoch 23/100, Train Loss: 0.09157
Epoch 24/100, Train Loss: 0.07551
Epoch 25/100, Train Loss: 0.07503
Epoch 26/100, Train Loss: 0.07470
Epoch 27/100, Train Loss: 0.07455
Epoch 28/100, Train Loss: 0.07431
Epoch 29/100, Train Loss: 0.07403


In [ ]:
# 만들긴 했는데, 굳이 안써도 될것 같다. 
def pre_eval(model, pre_test_dataset, device):
    pre_raw_eval_loader = torch.utils.data.DataLoader(
            pre_raw_test_dataset,
            batch_size = config["optimizer"]["pre_batch_size"],
            shuffle=False, # we dont need this in evaluation step => repeatable result, for comparision
            drop_last=False # to use all data
        )
    
    pre_raw_model.to(device)
    pre_raw_model.eval()
    
    eval_losses = []
    
    for x in pre_raw_eval_loader: 
        with torch.no_grad(): # no autograd => in evaluation step we dont need to update weights
            inputs = batch[0].to(device)
            reconstructed, _ = pre_raw_model(inputs)
            evel_loss = criterion(reconstructed,inputs)
            eval_losses.append(evel_loss.item())
    return np.mean(eval_losses)

In [71]:
# evaluation MDA

pre_raw_eval_loader = torch.utils.data.DataLoader(
        pre_raw_test_dataset,
        batch_size = config["optimizer"]["pre_batch_size"],
        shuffle=False, # we dont need this in evaluation step => repeatable result, for comparision
        drop_last=False # to use all data
    )

pre_raw_model.to(device)
pre_raw_model.eval()

eval_losses = []

for x in pre_raw_eval_loader: 
    with torch.no_grad(): # no autograd => in evaluation step we dont need to update weights
        inputs = batch[0].to(device)
        reconstructed, _ = pre_raw_model(inputs)
        evel_loss = criterion(reconstructed,inputs)
        eval_losses.append(evel_loss.item())

final_eval_loss = np.mean(eval_losses)
print(f"Final MSE of Model: {final_eval_loss:.5f}")

Final MSE of Model: 0.07000


## Fine tune

In [16]:
# we dont use train dataset here! we use latent features from pre-training.
# actually, this step is unnecessary. pre-fine both uses same data.
fine_raw_train_dataset, fine_raw_val_dataset, fine_raw_test_dataset, fine_raw_dims = get_data_corr(n_fold = config["env"]["fold"],
                                                           fp_radius = config["features"]["fp_radius"], 
                                                           transform_into_corr = False,
                                                           typ = ("rnaseq", "proteomics", "mutations", "methylations"),
                                                           #reconstructed = recon_sim
                                                                                   )

[20:20:38] DEPRECATION WARNING: please use MorganGenerator
[20:20:38] DEPRECATION WARNING: please use MorganGenerator
[20:20:38] DEPRECATION WARNING: please use MorganGenerator
[20:20:38] DEPRECATION WARNING: please use MorganGenerator
[20:20:38] DEPRECATION WARNING: please use MorganGenerator
[20:20:38] DEPRECATION WARNING: please use MorganGenerator
[20:20:38] DEPRECATION WARNING: please use MorganGenerator
[20:20:38] DEPRECATION WARNING: please use MorganGenerator
[20:20:38] DEPRECATION WARNING: please use MorganGenerator
[20:20:38] DEPRECATION WARNING: please use MorganGenerator
[20:20:38] DEPRECATION WARNING: please use MorganGenerator
[20:20:38] DEPRECATION WARNING: please use MorganGenerator
[20:20:38] DEPRECATION WARNING: please use MorganGenerator
[20:20:38] DEPRECATION WARNING: please use MorganGenerator
[20:20:38] DEPRECATION WARNING: please use MorganGenerator
[20:20:38] DEPRECATION WARNING: please use MorganGenerator
[20:20:38] DEPRECATION WARNING: please use MorganGenerat

In [78]:
#weight_path_raw = "trained_models/pretrained_raw_omics.pth"
_, main_raw_model = finetune_model_train(config,
                                         torch.utils.data.ConcatDataset([fine_raw_train_dataset, fine_raw_val_dataset]), 
                                         pre_trained_model = pretrained_model,
                                         use_momentum=False)
metrics = torchmetrics.MetricTracker(torchmetrics.MetricCollection(
    {"R_cellwise_residuals":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="drugs",
                          average="macro",
                          residualize=True),
    "R_cellwise":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="cell_lines",
                          average="macro",
                          residualize=False),
    "MSE":torchmetrics.MeanSquaredError()}))
metrics.to(device)
fine_raw_test_dataloader = torch.utils.data.DataLoader(fine_raw_test_dataset,
                                       batch_size=config["optimizer"]["batch_size"],
                                       drop_last=False,
                                      shuffle=False,
                                      pin_memory=True)

the omics data of 737 samples are replaced to pre-trained latent features.


/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: FutureWarning: The default value for `maximize` will be changed from `True` to `None` in v1.7.0 of TorchMetrics,will automatically infer the value based on the `higher_is_better` attribute of the metric (if such attribute exists) or raise an error if it does not. If you are explicitly setting the `maximize` argument to either `True` or `False` already, you can ignore this warning.
  warnings.warn(*args, **kwargs)  # noqa: B028


epoch : 0: train loss: 2.22680638850251 Smoothed R interaction (validation) None
epoch : 1: train loss: 1.8836918908674432 Smoothed R interaction (validation) None
epoch : 2: train loss: 1.819210430010877 Smoothed R interaction (validation) None
epoch : 3: train loss: 1.7625145434743836 Smoothed R interaction (validation) None
epoch : 4: train loss: 1.7220535630496088 Smoothed R interaction (validation) None
epoch : 5: train loss: 1.7004863274259532 Smoothed R interaction (validation) None
epoch : 6: train loss: 1.6739621645735279 Smoothed R interaction (validation) None
epoch : 7: train loss: 1.665813243212305 Smoothed R interaction (validation) None
epoch : 8: train loss: 1.6423299567513767 Smoothed R interaction (validation) None
epoch : 9: train loss: 1.633694402514341 Smoothed R interaction (validation) None
epoch : 10: train loss: 1.6285162163605944 Smoothed R interaction (validation) None
epoch : 11: train loss: 1.610349529163799 Smoothed R interaction (validation) None
epoch : 

In [79]:
main_raw_final_metrics_nohyper = scripts.evaluate_step(main_raw_model, fine_raw_test_dataloader, metrics, device, save_predictions = True, model_name = "pretrained_NoOptHyper", dataset_name = "raw_data")
print(f"main model final metrics: {main_raw_final_metrics_nohyper}")

Predictions saved to: results/pred_pretrained_NoOptHyper_raw_data_20250303_01:07:22_280646e8-3475-47d3-8114-c4166b3a2430.csv
main model final metrics: {'MSE': 1.7892388105392456, 'R_cellwise': 0.8867660164833069, 'R_cellwise_residuals': 0.21353977918624878}


# Hyperparameter Optimization

In [80]:
def train_model_optuna(trial, config):
    """
    Optuna를 활용한 하이퍼파라미터 최적화 함수
    - Pre-training (Autoencoder)
    - Fine-tuning (ResNet)
    """

    ### 1️⃣ Pre-training 하이퍼파라미터 샘플링 ###
    config["model"]["embed_dim"] = trial.suggest_int("embed_dim", 64, 512)
    config["model"]["hidden_dim_encoders"] = trial.suggest_int("hidden_dim_encoders", 64, 2048)
    config["model"]["fusion_dim"] = trial.suggest_int("fusion_dim", 64, 2048)
    config["model"]["dropout_encoders"] = trial.suggest_float("dropout_encoders", 0.0, 0.5)
    config["model"]["pre_training_epochs"] = trial.suggest_int("pre_training_epochs", 1, 500)
    config["optimizer"]["pre_batch_size"] = trial.suggest_int("pre_batch_size", 128, 512)
    config["optimizer"]["lr_pretraining"] = trial.suggest_float("lr_pretraining", 1e-6, 1e-1, log=True)

    ### 2️⃣ Pre-training 실행 ###
    try:
        #pre_raw_dataset = torch.utils.data.ConcatDataset([pre_raw_train_dataset, pre_raw_val_dataset])
        pre_trained_model, pre_training_losses = pre_train_step(config, pre_raw_train_dataset, input_dims=pre_raw_dims)

        # Pre-training 동안의 최소 loss 반환 (낮을수록 좋음)
        pre_training_score = min(pre_training_losses)
        print(f"Trial {trial.number}: Pre-training Loss = {pre_training_score:.5f}")

    except Exception as e:
        print(f"Pre-training Error: {e}")
        return float("inf")  # Pre-training 실패 시, 최대 loss 반환

    ### 3️⃣ Fine-tuning 하이퍼파라미터 샘플링 ###
    config["model"]["dropout_omics"] = trial.suggest_float("dropout_omics", 0.0, 0.9)
    config["model"]["dropout_omics_finetuning"] = trial.suggest_float("dropout_omics_finetuning", 0.0, 0.9)

    ### 4️⃣ Fine-tuning 실행 (Latent Representation 생성 포함) ###
    try:
        fine_tuned_model, val_target = finetune_model_train(
            config, fine_raw_train_dataset, pre_trained_model, fine_raw_val_dataset
        )

        print(f"Trial {trial.number}: Fine-tuning R_cellwise_residuals = {val_target:.5f}")
        return val_target  # Fine-tuning 성능 반환 (높을수록 좋음)

    except Exception as e:
        print(f"Fine-tuning Error: {e}")
        return -float("inf")  # Fine-tuning 실패 시, 최소 성능 반환



In [81]:
# 1️⃣ Optuna를 사용한 Hyperparameter Optimization (Validation 데이터 필요)
if config["env"]["search_hyperparameters"]:
    study_name = "raw_pretrained"
    storage_name = f"sqlite:///studies/{study_name}.db"

    study = optuna.create_study(
        study_name=study_name,
        storage=storage_name,
        direction='maximize',  # Fine-tuning 성능을 최적화 (R_cellwise_residuals 최대화)
        load_if_exists=True,
        pruner=optuna.pruners.MedianPruner(n_startup_trials=30, n_warmup_steps=5, interval_steps=5)
    )

    study.optimize(lambda trial: train_model_optuna(trial, config), n_trials=40)

    best_config = study.best_params
    print("Best Hyperparameters:", best_config)

    # 최적의 하이퍼파라미터를 config에 반영
    config["model"]["embed_dim"] = best_config["embed_dim"]
    config["model"]["hidden_dim_encoders"] = best_config["hidden_dim_encoders"]
    config["model"]["fusion_dim"] = best_config["fusion_dim"]
    config["model"]["dropout_encoders"] = best_config["dropout_encoders"]
    config["model"]["pre_training_epochs"] = best_config["pre_training_epochs"]
    config["optimizer"]["pre_batch_size"] = best_config["pre_batch_size"]
    config["optimizer"]["lr_pretraining"] = best_config["lr_pretraining"]
    config["model"]["dropout_omics"] = best_config["dropout_omics"]
    config["model"]["dropout_omics_finetuning"] = best_config["dropout_omics_finetuning"]

# 2️⃣ 최적 하이퍼파라미터를 적용한 후, Training (Train + Validation 합쳐서 사용)
pre_raw_dataset = torch.utils.data.ConcatDataset([pre_raw_train_dataset, pre_raw_val_dataset])

# Pre-training 실행
pre_trained_model, pre_training_losses = pre_train_step(config, pre_raw_dataset, input_dims=pre_raw_dims)

# Fine-tuning 실행 (이제 Train + Validation을 합쳐서 사용)
fine_tuned_model, final_target = finetune_model_train(
    config, pre_raw_dataset, pre_trained_model, validation_dataset=None  # ✅ Validation 없이 학습
)

print(f"Final Training Complete! Final R_cellwise_residuals: {final_target:.5f}")



Expected input_dims: 4
Epoch 1/100, Train Loss: 0.47340
Epoch 2/100, Train Loss: 0.22392
Epoch 3/100, Train Loss: 0.18894
Epoch 4/100, Train Loss: 0.16475
Epoch 5/100, Train Loss: 0.14655
Epoch 6/100, Train Loss: 0.13096
Epoch 7/100, Train Loss: 0.11915
Epoch 8/100, Train Loss: 0.11128
Epoch 9/100, Train Loss: 0.10199
Epoch 10/100, Train Loss: 0.09705
Epoch 11/100, Train Loss: 0.09227
Epoch 12/100, Train Loss: 0.14269
Epoch 13/100, Train Loss: 0.09182
Epoch 14/100, Train Loss: 0.08695
Epoch 15/100, Train Loss: 0.08497
Epoch 16/100, Train Loss: 0.08385
Epoch 17/100, Train Loss: 0.08271
Epoch 18/100, Train Loss: 0.08183
Epoch 19/100, Train Loss: 0.08117
Epoch 20/100, Train Loss: 0.08038
Epoch 21/100, Train Loss: 0.08002
Epoch 22/100, Train Loss: 0.07956


KeyboardInterrupt: 

# Mülleimer

In [3]:
@lru_cache(maxsize=None)
def get_data_corr_with_filtering(n_fold = 0, fp_radius = 2, transform_into_corr = True, typ = ["rnaseq", "mutations", "cnvs"]):
    # drug
    smile_dict = pd.read_csv("data/smiles.csv", index_col=0)
    fp = scripts.FingerprintFeaturizer(R = fp_radius)
    drug_dict = fp(smile_dict.iloc[:, 1], smile_dict.iloc[:, 0])
    
    # loading all datasets
    driver_genes = pd.read_csv("data/driver_genes.csv").loc[:, "symbol"].dropna()

    rnaseq = pd.read_csv("data/rnaseq_normcount.csv", index_col=0)
    driver_columns = rnaseq.columns.isin(driver_genes)
    filtered_rna = rnaseq.loc[:, driver_columns]
    
    proteomics = pd.read_csv("data/proteomics.csv", index_col=0)
    
    mutation = pd.read_csv("data/binary_mutations.csv")
    mutation.columns = mutation.iloc[0]
    mutation = mutation.iloc[2:,:].set_index("gene_symbol")
    driver_columns = mutation.columns.isin(driver_genes)
    filtered_mut = mutation.loc[:, driver_columns]
    filtered_mut = filtered_mut.astype(float)

    methylations = pd.read_csv("data/methylations.csv",index_col = 0).sort_index(ascending = True)

    cnvs = pd.read_csv("data/copy_number_variations.csv",index_col= 0)

    # concatenate all dataset 
    # inner join based on index: model_ids with NaN are automatically filtered out 
    data_concat = pd.concat([filtered_rna, proteomics, filtered_mut, methylations, cnvs], axis=1, join='inner')
    
    
    # Filter data by common indices in all modalities
    filtered_rna = filtered_rna[filtered_rna.index.isin(data_concat.index)]
    proteomics = proteomics[proteomics.index.isin(data_concat.index)]
    filtered_mut = filtered_mut[filtered_mut.index.isin(data_concat.index)]
    methylations = methylations[methylations.index.isin(data_concat.index)]
    cnvs = cnvs[cnvs.index.isin(data_concat.index)]
    
    # Initialize cell_dict
    cell_dict = {}

    if not transform_into_corr:
        for cell in data_concat.index:
            # Initialize a sub-dictionary for each cell
            cell_dict[cell] = {}
            
            # Add data for each type specified in typ
            if "rnaseq" in typ:
                cell_dict[cell]["rnaseq"] = torch.Tensor(filtered_rna.loc[cell].to_numpy())
            if "proteomics" in typ:
                cell_dict[cell]["proteomics"] = torch.Tensor(proteomics.loc[cell].to_numpy())
            if "mutations" in typ:
                cell_dict[cell]["mutations"] = torch.Tensor(filtered_mut.loc[cell].to_numpy())
            if "methylations" in typ:
                cell_dict[cell]["methylations"] = torch.Tensor(methylations.loc[cell].to_numpy())
            if "cnvs" in typ:
                cell_dict[cell]["cnvs"] = torch.Tensor(cnvs.loc[cell].to_numpy())

    # GDSC
    GDSC1 = pd.read_csv("data/GDSC1.csv", index_col=0)
    # default, remove data where lines or drugs are missing:
    data = GDSC1.query("SANGER_MODEL_ID in @data_concat.index & DRUG_ID in @drug_dict.keys()")
    unique_cell_lines = data.loc[:, "SANGER_MODEL_ID"].unique()

    np.random.seed(420) # for comparibility, don't change it!
    np.random.shuffle(unique_cell_lines)
    folds = np.array_split(unique_cell_lines, 10)
    test_lines = folds[0] 
    train_idxs = list(range(10))
    train_idxs.remove(n_fold)
    np.random.seed(420)
    validation_idx = np.random.choice(train_idxs)
    train_idxs.remove(validation_idx)
    train_lines = np.concatenate([folds[idx] for idx in train_idxs])
    validation_lines = folds[validation_idx]
    test_lines = folds[n_fold] 
    
    if transform_into_corr:
        # ic50 filtering
        ic50_mat = data.pivot(index = 'SANGER_MODEL_ID', columns = 'DRUG_ID', values = 'LN_IC50')
        drug_nan_ratio = ic50_mat.isna().mean(axis=0) 
        cellline_nan_ratio = ic50_mat.isna().mean(axis=1)
        filtered_ic50 = ic50_mat.loc[cellline_nan_ratio < 0.3, drug_nan_ratio < 0.3]
        imputer = KNNImputer(n_neighbors=5)  # k-NN에서 k=5
        imputed_ic50 = pd.DataFrame(
            imputer.fit_transform(filtered_ic50),
            index=filtered_ic50.index,
            columns=filtered_ic50.columns)      
        t = imputed_ic50.median()
        binarized_ic50 = imputed_ic50.apply(lambda x: x.apply(lambda v: 1 if v <= t[x.name] else 0), axis=0)
        
        # index filtering, here only exp, mutation, cnv data are used
        cell_line_index = binarized_ic50.index.intersection(data_concat.index)
        ic50 = binarized_ic50.loc[cell_line_index]
        
        # train, val, test among filtered data
        # these are valid train_, val_ and test_data index
        train_lines = np.intersect1d(train_lines, ic50.index)
        valid_validation_lines = np.intersect1d(validation_lines, ic50.index)
        valid_test_lines = np.intersect1d(test_lines, ic50.index)
        
        n_train = len(train_lines)  
        n_val = len(valid_validation_lines)      
        n_test = len(valid_test_lines)
        
        # Precompute similarity matrices for each data type
        similarity_matrices = {}
        
        if "rnaseq" in typ:
            exp_com = np.corrcoef(np.vstack([filtered_rna.loc[train_lines], 
                                             filtered_rna.loc[valid_validation_lines], 
                                             filtered_rna.loc[valid_test_lines]]), rowvar=True)
            train = exp_com[:n_train, :n_train]
            val = exp_com[n_train:n_train+n_val, :n_train]
            test = exp_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["rnaseq"] = sim_combined
        
        if "proteomics" in typ:
            prot_com = np.corrcoef(np.vstack([proteomics.loc[train_lines], 
                                              proteomics.loc[valid_validation_lines], 
                                              proteomics.loc[valid_test_lines]]), rowvar=True)
            train = prot_com[:n_train, :n_train]
            val = prot_com[n_train:n_train+n_val, :n_train]
            test = prot_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["proteomics"] = sim_combined
        
        if "mutations" in typ:
            train_snp = filtered_mut.loc[train_lines].astype(bool)
            val_snp = filtered_mut.loc[valid_validation_lines].astype(bool)
            test_snp = filtered_mut.loc[valid_test_lines].astype(bool)
            
            train = 1 - pairwise_distances(train_snp.values, train_snp.values, metric="jaccard")
            val = 1 - pairwise_distances(val_snp.values, train_snp.values, metric="jaccard")
            test = 1 - pairwise_distances(test_snp.values, train_snp.values, metric="jaccard")
    
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["mutations"] = sim_combined
        
        if "methylations" in typ:
            methyl_com = np.corrcoef(np.vstack([methylations.loc[train_lines], 
                                                methylations.loc[valid_validation_lines], 
                                                methylations.loc[valid_test_lines]]), rowvar=True)
            train = methyl_com[:n_train, :n_train]
            val = methyl_com[n_train:n_train+n_val, :n_train]
            test = methyl_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["mathylations"] = sim_combined
        
        if "cnvs" in typ:
            cnv_com = np.corrcoef(np.vstack([cnvs.loc[train_lines], 
                                             cnvs.loc[valid_validation_lines], 
                                             cnvs.loc[valid_test_lines]]), rowvar=True)
            train= cnv_com[:n_train, :n_train]
            val= cnv_com[n_train:n_train+n_val, :n_train]
            test= cnv_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["cnvs"] = sim_combined
            
        cell_dict = {}

        # 
        for cell in cell_line_index:
            cell_dict[cell] = {}
            for data_type in typ:
                sim_matrices = similarity_matrices[data_type]
                sim_tensor = torch.Tensor(sim_matrices)
                cell_idx = cell_line_index.get_loc(cell)
                cell_dict[cell][data_type] = sim_tensor[cell_idx]
                
        train_lines = train_lines
        validation_lines = valid_validation_lines
        test_lines = valid_test_lines

    # no change needed, query works fine with some missing
    train_data = data.query("SANGER_MODEL_ID in @train_lines")
    validation_data = data.query("SANGER_MODEL_ID in @validation_lines")
    test_data = data.query("SANGER_MODEL_ID in @test_lines")
    
    return (OmicsDataset_dict(cell_dict, drug_dict, train_data),
    OmicsDataset_dict(cell_dict, drug_dict, validation_data),
    OmicsDataset_dict(cell_dict, drug_dict, test_data))

In [17]:
n_fold = 0
fp_radius = 2
transform_into_corr = True
typ = ["rnaseq", "proteomics", "mutations", "methylations"]

# drug
smile_dict = pd.read_csv("data/smiles.csv", index_col=0)
fp = scripts.FingerprintFeaturizer(R = fp_radius)
drug_dict = fp(smile_dict.iloc[:, 1], smile_dict.iloc[:, 0])

# loading all datasets
driver_genes = pd.read_csv("data/driver_genes.csv").loc[:, "symbol"].dropna()

rnaseq = pd.read_csv("data/rnaseq_normcount.csv", index_col=0)
driver_columns = rnaseq.columns.isin(driver_genes)
filtered_rna = rnaseq.loc[:, driver_columns]

proteomics = pd.read_csv("data/proteomics.csv", index_col=0)

mutation = pd.read_csv("data/binary_mutations.csv")
mutation.columns = mutation.iloc[0]
mutation = mutation.iloc[2:,:].set_index("gene_symbol")
driver_columns = mutation.columns.isin(driver_genes)
filtered_mut = mutation.loc[:, driver_columns]
filtered_mut = filtered_mut.astype(float)

methylations = pd.read_csv("data/methylations.csv",index_col = 0).sort_index(ascending = True)

cnvs = pd.read_csv("data/copy_number_variations.csv",index_col= 0)

# concatenate all dataset 
# inner join based on index: model_ids with NaN are automatically filtered out 
data_concat = pd.concat([filtered_rna, proteomics, filtered_mut, methylations, cnvs], axis=1, join='inner')


# Filter data by common indices in all modalities
filtered_rna = filtered_rna[filtered_rna.index.isin(data_concat.index)]
proteomics = proteomics[proteomics.index.isin(data_concat.index)]
filtered_mut = filtered_mut[filtered_mut.index.isin(data_concat.index)]
methylations = methylations[methylations.index.isin(data_concat.index)]
cnvs = cnvs[cnvs.index.isin(data_concat.index)]

# Initialize cell_dict
cell_dict = {}

if not transform_into_corr:
    for cell in data_concat.index:
        # Initialize a sub-dictionary for each cell
        concatenated_data = []
        
        # Add data for each type specified in typ
        if "rnaseq" in typ:
            concatenated_data.append(filtered_rna.loc[cell].to_numpy())
        if "proteomics" in typ:
            concatenated_data.append(proteomics.loc[cell].to_numpy())
        if "mutations" in typ:
            concatenated_data.append(filtered_mut.loc[cell].to_numpy())
        if "methylations" in typ:
            concatenated_data.append(methylations.loc[cell].to_numpy())
        if "cnvs" in typ:
            concatenated_data.append(cnvs.loc[cell].to_numpy())

        cell_dict[cell] = torch.Tensor(np.concatenate(concatenated_data))

# GDSC
GDSC1 = pd.read_csv("data/GDSC1.csv", index_col=0)
# default, remove data where lines or drugs are missing:
data = GDSC1.query("SANGER_MODEL_ID in @data_concat.index & DRUG_ID in @drug_dict.keys()")
unique_cell_lines = data.loc[:, "SANGER_MODEL_ID"].unique()

np.random.seed(420) # for comparibility, don't change it!
np.random.shuffle(unique_cell_lines)
folds = np.array_split(unique_cell_lines, 10)
test_lines = folds[0] 
train_idxs = list(range(10))
train_idxs.remove(n_fold)
np.random.seed(420)
validation_idx = np.random.choice(train_idxs)
train_idxs.remove(validation_idx)
train_lines = np.concatenate([folds[idx] for idx in train_idxs])
validation_lines = folds[validation_idx]
test_lines = folds[n_fold] 

    # no change needed, query works fine with some missing
train_data = data.query("SANGER_MODEL_ID in @train_lines")
validation_data = data.query("SANGER_MODEL_ID in @validation_lines")
test_data = data.query("SANGER_MODEL_ID in @test_lines")

if transform_into_corr:
    # train, val, test among filtered data
    # these are valid train_, val_ and test_data index
    
    
    n_train = len(train_lines)  
    n_val = len(validation_lines)      
    n_test = len(test_lines)
    
    # Precompute similarity matrices for each data type
    similarity_matrices = {}
    
    if "rnaseq" in typ:
        exp_com = np.corrcoef(np.vstack([filtered_rna.loc[train_lines], 
                                         filtered_rna.loc[validation_lines], 
                                         filtered_rna.loc[test_lines]]), rowvar=True)
        train = exp_com[:n_train, :n_train]
        val = exp_com[n_train:n_train+n_val, :n_train]
        test = exp_com[n_train+n_val:, :n_train]
        sim_combined = np.vstack([train, val, test])
        similarity_matrices["rnaseq"] = sim_combined
    
    if "proteomics" in typ:
        prot_com = np.corrcoef(np.vstack([proteomics.loc[train_lines], 
                                          proteomics.loc[validation_lines], 
                                          proteomics.loc[test_lines]]), rowvar=True)
        train = prot_com[:n_train, :n_train]
        val = prot_com[n_train:n_train+n_val, :n_train]
        test = prot_com[n_train+n_val:, :n_train]
        sim_combined = np.vstack([train, val, test])
        similarity_matrices["proteomics"] = sim_combined
    
    if "mutations" in typ:
        train_snp = filtered_mut.loc[train_lines].astype(bool)
        val_snp = filtered_mut.loc[validation_lines].astype(bool)
        test_snp = filtered_mut.loc[test_lines].astype(bool)
        
        train = 1 - pairwise_distances(train_snp.values, train_snp.values, metric="jaccard")
        val = 1 - pairwise_distances(val_snp.values, train_snp.values, metric="jaccard")
        test = 1 - pairwise_distances(test_snp.values, train_snp.values, metric="jaccard")

        sim_combined = np.vstack([train, val, test])
        similarity_matrices["mutations"] = sim_combined
    
    if "methylations" in typ:
        methyl_com = np.nan_to_num(np.corrcoef(np.vstack([methylations.loc[train_lines], 
                                            methylations.loc[validation_lines], 
                                            methylations.loc[test_lines]]), rowvar=True))
        train = methyl_com[:n_train, :n_train]
        val = methyl_com[n_train:n_train+n_val, :n_train]
        test = methyl_com[n_train+n_val:, :n_train]
        sim_combined = np.vstack([train, val, test])
        similarity_matrices["methylations"] = sim_combined
    
    if "cnvs" in typ:
        cnv_com = np.nan_to_num(np.corrcoef(np.vstack([cnvs.loc[train_lines], # nan-generation problem fixed 
                                         cnvs.loc[validation_lines], 
                                         cnvs.loc[test_lines]]), rowvar=True))
        train= cnv_com[:n_train, :n_train]
        val= cnv_com[n_train:n_train+n_val, :n_train]
        test= cnv_com[n_train+n_val:, :n_train]
        sim_combined = np.vstack([train, val, test])
        similarity_matrices["cnvs"] = sim_combined
        
    cell_dict = {}

    # 
    for cell in unique_cell_lines:
        cell_dict[cell] = {}
        for data_type in typ:
            sim_matrices = similarity_matrices[data_type]
            sim_tensor = torch.Tensor(sim_matrices)
            cell_idx = np.where(unique_cell_lines == cell)[0][0]
            cell_dict[cell][data_type] = sim_tensor[cell_idx]

[14:39:47] DEPRECATION WARNING: please use MorganGenerator
[14:39:47] DEPRECATION WARNING: please use MorganGenerator
[14:39:47] DEPRECATION WARNING: please use MorganGenerator
[14:39:47] DEPRECATION WARNING: please use MorganGenerator
[14:39:47] DEPRECATION WARNING: please use MorganGenerator
[14:39:47] DEPRECATION WARNING: please use MorganGenerator
[14:39:47] DEPRECATION WARNING: please use MorganGenerator
[14:39:47] DEPRECATION WARNING: please use MorganGenerator
[14:39:47] DEPRECATION WARNING: please use MorganGenerator
[14:39:47] DEPRECATION WARNING: please use MorganGenerator
[14:39:47] DEPRECATION WARNING: please use MorganGenerator
[14:39:47] DEPRECATION WARNING: please use MorganGenerator
[14:39:47] DEPRECATION WARNING: please use MorganGenerator
[14:39:47] DEPRECATION WARNING: please use MorganGenerator
[14:39:47] DEPRECATION WARNING: please use MorganGenerator
[14:39:47] DEPRECATION WARNING: please use MorganGenerator
[14:39:47] DEPRECATION WARNING: please use MorganGenerat

In [21]:
len(similarity_matrices["methylations"][0])

587

In [28]:
np.isnan(similarity_matrices["methylations"]).any()

np.True_

In [31]:
methylations.to_csv("met_nan_check.csv")

In [36]:
type(reconstructed)

torch.Tensor

In [40]:
torch.save(reconstructed,"reconstructedMDA_data.pth")

In [53]:
data_concat.shape

(737, 3996)

In [82]:
len(fine_sim_train_dataset[0][0])/4

587.0

In [85]:
sim_combined.shape[0]*4 # 왜 2348이 나오지? 이 디멘션 문제를 해결해야한다. 

2940

In [36]:
len(fine_sim_train_dataset[0][0]) # 정상적으로 4개의 sim mat이 concat되었다면 2940이 나와야한다. 이것부터 해결하고, 트레이닝 돌려보자.

2348

In [38]:
len(batch[0])

200

In [34]:
weights = torch.load("trained_models/pretrained_omics.pth")

In [33]:
for key, value in weights.items():
    print(f"{key}: {value.shape}")

0.weight: torch.Size([700, 1940])
0.bias: torch.Size([700])
2.weight: torch.Size([485, 700])
2.bias: torch.Size([485])


In [56]:
len(filtered_rna.loc[cell].to_numpy())

777

In [57]:
len(proteomics.loc[cell].to_numpy())

1642

In [58]:
len(filtered_mut.loc[cell].to_numpy())

774

In [60]:
len(methylations.loc[cell].to_numpy())

378

In [62]:
len(similarity_matrices["rnaseq"][0])

587